In [ ]:
"""e-ORP"""

# Copyright (c) 2025 Doug Currie

import ipywidgets as widgets
import pandas as pd
import plotly.express as px
from IPython.display import display, Markdown
import pyscipopt

pd.options.display.max_columns = None # don't limit number of displayed columns
pd.options.display.precision = 3      # display up to 3 decimal places in dataframes

# ############## Input Widgets ##############

#	**Set Model Parameters**

byear_box = widgets.BoundedIntText(
    value=2024,
    min=2000,
    max=9999,
    step=1,
    description='Base Year:',
    disabled=False
)
pyear_box = widgets.BoundedIntText(
    value=30,
    min=1,
    max=100,
    step=1,
    description='Plan Years:',
    disabled=False
)
rors_box = widgets.BoundedFloatText(
    value=0.07,
    min=0.0,
    max=0.99,
    step=0.01,
    description='ROR Stocks',
    disabled=False
)
rorb_box = widgets.BoundedFloatText(
    value=0.03,
    min=0.0,
    max=0.99,
    step=0.01,
    description='ROR Bonds',
    disabled=False
)
fras_box = widgets.BoundedFloatText(
    value=0.60,
    min=0.0,
    max=1.00,
    step=0.01,
    description='Stock Ratio',
    disabled=False
)
frab_box = widgets.BoundedFloatText(
    value=0.40,
    min=0.0,
    max=1.00,
    step=0.01,
    description='Bonds Ratio',
    disabled=False
)
infl_box = widgets.BoundedFloatText(
    value=0.02,
    min=0.0,
    max=0.99,
    step=0.01,
    description='Inflation',
    disabled=False
)
infs_box = widgets.BoundedFloatText(
    value=0.04,
    min=0.0,
    max=0.99,
    step=0.01,
    description='SpendRate',
    disabled=False
)
incn_box = widgets.BoundedFloatText(
    value=50,
    min=0.0,
    max=999.9,
    step=1.0,
    description='Spending $',
    disabled=False
)
xinc_box = widgets.BoundedFloatText(
    value=1,
    min=0.0,
    max=999.9,
    step=1,
    description='Misc. Income $',
    disabled=False
)
xinr_box = widgets.BoundedFloatText(
    value=0.00,
    min=-1.0,
    max=0.99,
    step=0.01,
    description='Misc. Inc. δ',# Δ
    disabled=False
)
# paired values for spouses
#
aage1_box = widgets.BoundedIntText(
    value=65,
    min=0,
    max=149,
    step=1,
    description='Age:',
    disabled=False
)
aage2_box = widgets.BoundedIntText(
    value=65,
    min=0,
    max=149,
    step=1,
    description='Age:',
    disabled=False
)
atax1_box = widgets.BoundedFloatText(
    value=50.0,
    min=0,
    max=9999.9,
    step=1.0,
    description='After Tax $:',
    disabled=False
)
atax2_box = widgets.BoundedFloatText(
    value=50.0,
    min=0,
    max=9999.9,
    step=1.0,
    description='After Tax $:',
    disabled=False
)
bsis1_box = widgets.BoundedFloatText(
    value=10.0,
    min=0,
    max=9999.9,
    step=1.0,
    description='Cost Basis $:',
    disabled=False
)
bsis2_box = widgets.BoundedFloatText(
    value=10.0,
    min=0,
    max=9999.9,
    step=1.0,
    description='Cost Basis $:',
    disabled=False
)
taxd1_box = widgets.BoundedFloatText(
    value=100.0,
    min=0,
    max=9999.9,
    step=1.0,
    description='Trad IRA $:',
    disabled=False
)
taxd2_box = widgets.BoundedFloatText(
    value=100.0,
    min=0,
    max=9999.9,
    step=1.0,
    description='Trad IRA $:',
    disabled=False
)
roth1_box = widgets.BoundedFloatText(
    value=100.0,
    min=0,
    max=9999.9,
    step=1.0,
    description='Roth IRA $:',
    disabled=False
)
roth2_box = widgets.BoundedFloatText(
    value=100.0,
    min=0,
    max=9999.9,
    step=1.0,
    description='Roth IRA $:',
    disabled=False
)
ssar1_box = widgets.BoundedFloatText(
    value=36.0,
    min=0,
    max=9999.9,
    step=1.0,
    description='SSA/year $:',
    disabled=False
)
ssar2_box = widgets.BoundedFloatText(
    value=36.0,
    min=0,
    max=9999.9,
    step=1.0,
    description='SSA/year $:',
    disabled=False
)
refa1_box = widgets.BoundedIntText(
    value=65,
    min=62,
    max=99,
    step=1,
    description='Ref. Age:',
    disabled=False
)
refa2_box = widgets.BoundedIntText(
    value=65,
    min=62,
    max=99,
    step=1,
    description='Ref. Age:',
    disabled=False
)
reta1_box = widgets.BoundedIntText(
    value=70,
    min=62,
    max=70,
    step=1,
    description='Claim Age:',
    disabled=False
)
reta2_box = widgets.BoundedIntText(
    value=65,
    min=62,
    max=70,
    step=1,
    description='Claim Age:',
    disabled=False
)
fstat_box = widgets.Dropdown(
    options=[('Single', 0), ('Married Filing Jointly', 1), ('Head of Household', 2)],
    value=1,
    disabled=False,
    description='Filing Status:'
)
# csv file load/save name and buttons
#
pfname = widgets.Text(
    value='params/fname.csv',
    placeholder='filename.csv',
    description='File Name:',
    disabled=False
)
save_button = widgets.Button(
    description='Save Parameters Button',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Save Parameters',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)
load_button = widgets.Button(
    description='Load Parameters Button',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Load Parameters',
    icon='upload' # (FontAwesome names without the `fa-` prefix)
)

def load_params(_):
    """Read params from csv file and load into widgets"""
    ps = pd.read_csv(filepath_or_buffer=pfname.value, index_col=0)
    byear_box.value = ps.loc['byear']['0']
    pyear_box.value = ps.loc['pyear']['0']
    rors_box.value =  ps.loc['rors' ]['0']
    rorb_box.value =  ps.loc['rorb' ]['0']
    fras_box.value =  ps.loc['fras' ]['0']
    frab_box.value =  ps.loc['frab' ]['0']
    infl_box.value =  ps.loc['infl' ]['0']
    infs_box.value =  ps.loc['infs' ]['0']
    incn_box.value =  ps.loc['incn' ]['0']
    xinc_box.value =  ps.loc['xinc' ]['0']
    xinr_box.value =  ps.loc['xinr' ]['0']
    aage1_box.value = ps.loc['aage1']['0']
    aage2_box.value = ps.loc['aage2']['0']
    atax1_box.value = ps.loc['atax1']['0']
    atax2_box.value = ps.loc['atax2']['0']
    bsis1_box.value = ps.loc['bsis1']['0']
    bsis2_box.value = ps.loc['bsis2']['0']
    taxd1_box.value = ps.loc['taxd1']['0']
    taxd2_box.value = ps.loc['taxd2']['0']
    roth1_box.value = ps.loc['roth1']['0']
    roth2_box.value = ps.loc['roth2']['0']
    ssar1_box.value = ps.loc['ssar1']['0']
    ssar2_box.value = ps.loc['ssar2']['0']
    refa1_box.value = ps.loc['refa1']['0']
    refa2_box.value = ps.loc['refa2']['0']
    reta1_box.value = ps.loc['reta1']['0']
    reta2_box.value = ps.loc['reta2']['0']
    fstat_box.value = ps.loc['fstat']['0']
    return ps

def save_params(_):
    """Save params to csv file from widgets"""
    idx = [
		'byear',
		'pyear',
		'rors' ,
		'rorb' ,
		'fras' ,
		'frab' ,
		'infl' ,
		'infs' ,
		'incn' ,
        'xinc' ,
        'xinr' ,
		'aage1',
		'aage2',
		'atax1',
		'atax2',
		'bsis1',
		'bsis2',
		'taxd1',
		'taxd2',
		'roth1',
		'roth2',
		'ssar1',
		'ssar2',
		'refa1',
		'refa2',
		'reta1',
		'reta2',
        'fstat'
	]
    val = [
		byear_box.value,
		pyear_box.value,
		rors_box.value,
		rorb_box.value,
		fras_box.value,
		frab_box.value,
		infl_box.value,
		infs_box.value,
		incn_box.value,
        xinc_box.value,
        xinr_box.value,
		aage1_box.value,
		aage2_box.value,
		atax1_box.value,
		atax2_box.value,
		bsis1_box.value,
		bsis2_box.value,
		taxd1_box.value,
		taxd2_box.value,
		roth1_box.value,
		roth2_box.value,
		ssar1_box.value,
		ssar2_box.value,
		refa1_box.value,
		refa2_box.value,
		reta1_box.value,
		reta2_box.value,
        fstat_box.value
	]
    ps = pd.Series(index=idx, data=val)
    ps.to_csv(path_or_buf=pfname.value)
    return ps

save_button.on_click(save_params)
load_button.on_click(load_params)

becenter = widgets.Layout(display='flex', justify_content='center')

winputs = widgets.VBox([
    widgets.GridBox([widgets.Label('e-ORP', style={
                        'font_weight':'bold',
                        'font_size':'large',
                        'text_color':'forestgreen',
                        }), widgets.Label(''),
                    widgets.Label('Set Model Parameters', style={
                        'font_weight':'bold',
                        'font_size':'large',
                        }),
                 widgets.Label(''),
                 widgets.Label('Note: All dollar values in 000s...', style={'font_style':'italic',}),
                 widgets.Label(''),
                 byear_box, widgets.Label('account values below are at the end of the Base Year'),
                 pyear_box, widgets.Label('number of years to include in plan'),
                 rors_box, widgets.Label('rate of return on stock portion of accounts'),
                 rorb_box, widgets.Label('rate of return on bond portion of accounts'),
                 fras_box, widgets.Label('fraction of portfolio in stock investments'),
                 frab_box, widgets.Label('fraction of portfolio in bond investments'),
                 infl_box, widgets.Label('annual inflation rate for SSA payments & tax brackets'),
                 infs_box, widgets.Label('annual inflation rate applied to spending'),
                 incn_box, widgets.Label('annual income needed for spending, after taxes'),
                 xinc_box, widgets.Label('miscellaneous annual income (Inherited IRA, etc.)'),
                 xinr_box, widgets.Label('rate of change (+/-) applied to the miscellaneous income'),
                 fstat_box, widgets.Label('Federal Income Tax filing status'),
                ],
                layout=widgets.Layout(grid_template_columns='35% 65%')),

    widgets.GridBox([
                 widgets.Label('Person', layout=becenter),
                 widgets.Label('Spouse', layout=becenter),
                 widgets.Label(''),
                 aage1_box, aage2_box, widgets.Label('Age at end of base year'),
                 atax1_box, atax2_box, widgets.Label('Non-retirement accounts'),
                 bsis1_box, bsis2_box, widgets.Label('Cost basis of above (for cap gains calc)'),
                 taxd1_box, taxd2_box, widgets.Label('Tax Deferred IRAs and 401(k)s'),
                 roth1_box, roth2_box, widgets.Label('Roth IRAs and 401(k)s'),
                 ssar1_box, ssar2_box, widgets.Label('Guaranteed (SSA) income, annual'),
                 refa1_box, refa2_box, widgets.Label('Reference age of guaranteed income'),
                 reta1_box, reta2_box, widgets.Label('Age when SSA is claimed')
                ],
                layout=widgets.Layout(grid_template_columns='35% 35% 30%')),

    widgets.HBox([pfname, save_button, load_button])
])


# ############## Error Output Widget ##############

# Use this for displaying error outputs
err_out = widgets.Output(layout={'border': '1px solid black'})

# ##############      Tax Data       ##############

# Income tax rates

tax_rates =    [ 0.100,   0.120,   0.220,   0.240,   0.320,   0.350,   0.370 ]

# Data for 2025; update by applying inflation rate for projections
# each entry in the ordered dict is top of income bracket in 000s

tax_brackets = [[11.925, 48.475, 103.350, 197.300, 250.525, 626.350], # Single
                [23.850, 96.950, 206.700, 394.600, 501.050, 751.600], # MFJ Married Filing Jointly
                [17.000, 64.850, 103.350, 197.300, 250.500, 626.350]] # Head of Household

# Capital Gains tax rates

cgt_rates =     [0.000, 0.150, 0.200]

cgt_brackets =  [[48.35, 600.05], # Single
                 [96.70, 533.40], # MFJ Married Filing Jointly
                 [64.75, 566.70]] # Head of Household

std_deductions = [15.750, 31.500, 23.125] # Single, MFJ, HoH

# additional standard deduction based on age >= 65 or blindness is $1,600 
#
# OBBBA adds an additional $6,000 per person deduction based on age >= 65 through 2028
# taxpayers with modified adjusted gross income over $75,000 ($150,000 for joint filers).
#
addl_obbba_deduction_age65 = 6.000
addl_deduction_age65 = 1.600

base_year_irs_brackets = 2025

# filing_status: 0: Single, 1: MFJ, 2: Head of Household
#
def tax_bucket_n_size(year, n, age1, age2, filing_status=1, rate_infla=0.02):
    infla = 1.0
    if year < base_year_irs_brackets:
        with err_out:
            print('FAIL: no tax info for years before 2025')
        year = base_year_irs_brackets
    else:
        infla = (1.0 + rate_infla) ** (year - base_year_irs_brackets)
    size = 0.0 # the size of this tax bucket
    if n == 0:
        size = std_deductions[filing_status] * infla
        if age1 >= 65:
            size += addl_deduction_age65
            if year <= 2028: size += addl_obbba_deduction_age65
        if filing_status == 1 and age2 >= 65:
            size += addl_deduction_age65
            if year <= 2028: size += addl_obbba_deduction_age65
    elif n == 1:
        size = tax_brackets[filing_status][0] * infla
    else:
        size = (tax_brackets[filing_status][n-1] - tax_brackets[filing_status][n-2]) * infla
    return size

def cgt_bucket_n_size(year, n, filing_status=1, rate_infla=0.02):
    infla = 1.0
    if year < base_year_irs_brackets:
        with err_out:
            print('FAIL: no tax info for years before 2025')
        year = base_year_irs_brackets
    else:
        infla = (1.0 + rate_infla) ** (year - base_year_irs_brackets)
    size = 0.0 # the size of this tax bucket
    if n == 0:
        size = cgt_brackets[filing_status][0] * infla
    else:
        size = (tax_brackets[filing_status][1] - tax_brackets[filing_status][0]) * infla
    return size

#

RMD_divisor = [ 27.4, 26.5, 25.5, 24.6, 23.7, 22.9, 22.0, 21.1, 20.2, 19.4, 18.5,
                17.7, 16.8, 16.0, 15.2, 14.4, 13.7, 12.9, 12.2, 11.5, 10.8, 10.1,
                 9.5,  8.9,  8.4,  7.8,  7.3,  6.8,  6.4,  6.0,  5.6,  5.2,  4.9,
                 4.6,  4.3,  4.1,  3.9,  3.7,  3.5,  3.4,  3.3,  3.1,  3.0,  2.9,
                 2.8,  2.7,  2.5,  2.3,  2.0]
#

# ############## The Planning Dataframe ##############

def make_planning_datadict(reduce_SSAb):
    """Create the datadict to be used by OORPy, populated from the widgets"""
    byear = byear_box.value
    years = pyear_box.value + 1
    idx = range(byear, byear + years)

    def ssa_calc(y):
        """Calculate SSA annual income based on age and initial data from the widgets"""
        e = aage1_box.value + y
        j = aage2_box.value + y
        e_ssa = ssar1_box.value * ((1.0 + infl_box.value) ** (e - refa1_box.value)) if e > reta1_box.value else 0.0
        j_ssa = ssar2_box.value * ((1.0 + infl_box.value) ** (j - refa2_box.value)) if j >= reta2_box.value else 0.0
        return (j_ssa + e_ssa) * (0.77 if (reduce_SSAb and ((byear + y) >= 2035)) else 1.0)

    dd = {'e':              range(aage1_box.value, aage1_box.value + years),
          'j':              range(aage2_box.value, aage2_box.value + years),
          'e_RothConv' :    [0.0 for x in idx],
          'j_RothConv' :    [0.0 for x in idx],
          'e_RMD':          [0.0 for x in idx],
          'j_RMD':          [0.0 for x in idx],
          'SSA_income':     [ssa_calc(y) for y in range(len(idx))],
          'misc_income':    [xinc_box.value * (1.0 + xinr_box.value) ** y for y in range(len(idx))],
          'auto_income':    [0.0 for x in idx], # sum of previous four
          'taxable_income': [0.0 for x in idx],
            # 'irmaa_level':    [0   for x in idx],
          'dividends':      [0.0 for x in idx],
          'capgains':       [0.0 for x in idx],
          'income_reqd':    [incn_box.value * (1.0 + infs_box.value) ** y for y in range(len(idx))],
          'income_tax':     [0.0 for x in idx],
          'tax_bracket':    [0.0 for x in idx],
          'cgains_rate':    [0.0 for x in idx],
          'from_aTax':      [0.0 for x in idx],
          'from_eRoth':     [0.0 for x in idx],
          'from_jRoth':     [0.0 for x in idx],
          'from_eTaxd':     [0.0 for x in idx],
          'from_jTaxd':     [0.0 for x in idx],
          'afterTax':       [(atax1_box.value + atax2_box.value) for x in idx],
          'aTax_basis':     [(bsis1_box.value + bsis2_box.value) for x in idx],
          'e_Roth':         [roth1_box.value for x in idx],
          'e_Taxd':         [taxd1_box.value for x in idx],
          'j_Roth':         [roth2_box.value for x in idx],
          'j_Taxd':         [taxd2_box.value for x in idx],
           # Tax buckets
          'tax0':          [0.0 for x in idx],
          'tax1':          [0.0 for x in idx],
          'tax2':          [0.0 for x in idx],
          'tax3':          [0.0 for x in idx],
          'tax4':          [0.0 for x in idx],
          'tax5':          [0.0 for x in idx],
          'tax6':          [0.0 for x in idx],
          'cgt0':          [0.0 for x in idx],
          'cgt15':         [0.0 for x in idx],
          #
          'inflation':      [infs_box.value for y in idx],
          'net_pretax':     [0.0 for x in idx],
          'net_postax':     [0.0 for x in idx],
          'e_RMD_factor':   [(0.0 if e < 73 else 1 / RMD_divisor[e - 72])
                             for e in range(aage1_box.value, aage1_box.value + years)],
          'j_RMD_factor':   [(0.0 if j < 73 else 1 / RMD_divisor[j - 72])
                             for j in range(aage1_box.value, aage1_box.value + years)],
          'year':           idx,
        }

    # compute tax brackets -- TODO -- assumes AGI qualifies for OBBBA extra deduction
    for y in range(years):
        dd['tax0'][y] = tax_bucket_n_size(dd['year'][y], 0, dd['e'][y], dd['j'][y], fstat_box.value, infl_box.value)
        dd['tax1'][y] = tax_bucket_n_size(dd['year'][y], 1, dd['e'][y], dd['j'][y], fstat_box.value, infl_box.value)
        dd['tax2'][y] = tax_bucket_n_size(dd['year'][y], 2, dd['e'][y], dd['j'][y], fstat_box.value, infl_box.value)
        dd['tax3'][y] = tax_bucket_n_size(dd['year'][y], 3, dd['e'][y], dd['j'][y], fstat_box.value, infl_box.value)
        dd['tax4'][y] = tax_bucket_n_size(dd['year'][y], 4, dd['e'][y], dd['j'][y], fstat_box.value, infl_box.value)
        dd['tax5'][y] = tax_bucket_n_size(dd['year'][y], 5, dd['e'][y], dd['j'][y], fstat_box.value, infl_box.value)
        dd['tax6'][y] = tax_bucket_n_size(dd['year'][y], 6, dd['e'][y], dd['j'][y], fstat_box.value, infl_box.value)
        dd['cgt0'][y] = dd['tax0'][y] + cgt_bucket_n_size(dd['year'][y], 0, fstat_box.value, infl_box.value)
        dd['cgt15'][y] = cgt_bucket_n_size(dd['year'][y], 1, fstat_box.value, infl_box.value)

    return dd # was: pd.DataFrame(dd, index = idx)

# NLP

VARS = [
    # Initial Values for year 0
    'afterTax',
    'aTax_basis',
    'e_Roth',
    'e_Taxd',
    'j_Roth',
    'j_Taxd',
    # Configuration Values
    'income_reqd',   #  year 0 (for option A & B); year 1..N for option A
    # LP Vars for years 1..N
    'from_eRoth',
    'from_jRoth',
    'from_eTaxd',
    'from_jTaxd',
    'from_aTax',
    'to_aTax',
    # Intermediate Calculated values
    'e_RMD',
    'j_RMD',
    'auto_income', #  = e_RMD + j_RMD + SSA_income + misc_income
    'e_RothConv',
    'j_RothConv',
    'taxable_income',
    'dividends',
    'capgains',
    'tax0',  # 0% income tax bucket
    'tax1',  # next (10%) income tax bucket, ...
    'tax2',
    'tax3',
    'tax4',
    'tax5',
    'tax6',
    'tax7',
    'cgt0',  #  0% capital gains tax bucket
    'cgt15', # 15% capital gains tax bucket, ...
    'cgt20',
    'ncgt0', # 0% offset capital gains tax bucket (filled with ordinary income), ...
    'ncgt15',
    'ncgt20',
    'income_tax',
    'net_pretax',
    'net_postax'
]

def lop_to_cents(x):
    """Truncate model (float) data to 5 decimal digits"""
    if x == None:
        return -0.0 # unique value to identify unconstrained/unused values
    else:
        return max(0,round(x, 5))

def oorplp(dd, RothConvs, objective, ftab):
    """Run OORPyLP with specified objective, 'net_pretax', 'net_postax', 
        or a value for a specified residual with maximum spend
       RothConvs must be 0 to prevent, or 1 to enable conversions
    """
    err_out.clear_output() # at start of each run
    out_box.clear_output()
    # config values from UI
    frac_bonds = frab_box.value
    frac_stock = fras_box.value
    ror_bonds  = rorb_box.value
    ror_stock  = rors_box.value
    ror_investment = ror_stock * frac_stock + ror_bonds * frac_bonds
    rori = 1.0 + ror_investment
    rors = 1.0 + (frac_stock * ror_stock)
    rorb = 1.0 + (frac_bonds * ror_bonds)
    # with err_out:
    #     print(f'rori: {rori: 1.6f} rors: {rors: 1.6f} rorb: {rorb: 1.6f}')
    #     print(f'basis: {dd["aTax_basis"][0]: 3.3f}')
    #     print(f'rconv: {RothConvs}')
    # the model
    scip = pyscipopt.Model()
    # scip.setEmphasis(pyscipopt.SCIP_PARAMEMPHASIS.HARDLP) # ? NUMERICS, PHASEFEAS, CPSOLVER no help
    # set up problem
    YRS = len(dd['e']) - 1 # number of years of projection from base year 0
    IDX = range(0,YRS+1)   # 0 (base year) .. YRS (final year)
    vars = {}
    for v in VARS:
        vars[v] = {}
        for i in IDX:
            vars[v][i] = scip.addVar(vtype='C', name=f"Proje_{v}_{i}")
    # Objective
    if isinstance(objective, str):
        scip.setObjective(vars[objective][YRS], sense="maximize")
        # subject to:
        for y in range(1,YRS+1):
            scip.addCons(dd['income_reqd'][y] == vars['income_reqd'][y])
    else:
        scip.setObjective(vars['income_reqd'][0], sense="maximize") # 'Maximize Spend'
        # subject to growth and minimum residual:
        scip.addCons(vars['net_pretax'][YRS] >= ftab) # 'Minimum Residual'
        # and minimum income
        scip.addCons(vars['income_reqd'][0] >= dd['income_reqd'][0])
        for y in range(1,YRS+1):
            scip.addCons(vars['income_reqd'][y] == (1.0 + dd['inflation'][y]) * vars['income_reqd'][y-1])

    # Tuning Constraints
    for y in range(0,YRS+1):
        scip.chgVarLb(vars['afterTax'][y], 0.1)
    
    # Initial Values Constraints
    scip.addCons(vars['afterTax'][0] == dd['afterTax'][0])
    scip.addCons(vars['aTax_basis'][0] == dd['aTax_basis'][0])
    scip.addCons(vars['e_Roth'][0] == dd['e_Roth'][0])
    scip.addCons(vars['e_Taxd'][0] == dd['e_Taxd'][0])
    scip.addCons(vars['j_Roth'][0] == dd['j_Roth'][0])
    scip.addCons(vars['j_Taxd'][0] == dd['j_Taxd'][0])
    
    # Calculation Constraints
    for y in range(1,YRS+1):
        scip.addCons(vars['e_RMD'][y] == dd['e_RMD_factor'][y] * vars['e_Taxd'][y-1])
        scip.addCons(vars['j_RMD'][y] == dd['j_RMD_factor'][y] * vars['j_Taxd'][y-1])
        scip.addCons(vars['dividends'][y] == (rorb - 1.0) * vars['afterTax'][y-1])

        scip.addCons(vars['e_RothConv'][y] <= RothConvs * vars['e_Taxd'][y])
        scip.addCons(vars['j_RothConv'][y] <= RothConvs * vars['j_Taxd'][y])

        scip.addCons(vars['auto_income'][y] == vars['e_RMD'][y] + vars['j_RMD'][y] + vars['dividends'][y] \
                                                + dd['misc_income'][y] + dd['SSA_income'][y])
        
        scip.addCons(vars['taxable_income'][y] == vars['e_RMD'][y] + vars['j_RMD'][y] + vars['dividends'][y] \
                                                + dd["misc_income"][y] + 0.85 * dd["SSA_income"][y] \
                                                + vars['from_eTaxd'][y] + vars['from_jTaxd'][y] \
                                                + vars['e_RothConv'][y] + vars['j_RothConv'][y])

        # using a nonlinear solver for capgains
        #scip.addCons(vars['capgains'][y] == vars['from_aTax'][y] \
        #                                     * ((vars['afterTax'][y-1] - vars['aTax_basis'][y-1]) / vars['afterTax'][y-1]))
        scip.addCons((vars['capgains'][y] * vars['afterTax'][y-1]) \
                             == vars['from_aTax'][y] * (vars['afterTax'][y-1] - vars['aTax_basis'][y-1]))
        scip.addCons(vars['capgains'][y] <= vars['from_aTax'][y])

        scip.addCons(vars['aTax_basis'][y] == vars['aTax_basis'][y-1] \
                                             - (vars['from_aTax'][y] - vars['capgains'][y]) \
                                             + vars['to_aTax'][y])
        
        scip.addCons(vars['income_reqd'][y] == vars['auto_income'][y] + vars['from_aTax'][y] \
                                            + vars['from_eTaxd'][y] + vars['from_jTaxd'][y] \
                                            + vars['from_eRoth'][y] + vars['from_jRoth'][y] \
                                            - vars['income_tax'][y] - vars['to_aTax'][y])
        
        # TODO qualified dividends?

        scip.addCons(vars['tax0'][y] <= dd['tax0'][y])
        scip.addCons(vars['tax1'][y] <= dd['tax1'][y])
        scip.addCons(vars['tax2'][y] <= dd['tax2'][y])
        scip.addCons(vars['tax3'][y] <= dd['tax3'][y])
        scip.addCons(vars['tax4'][y] <= dd['tax4'][y])
        scip.addCons(vars['tax5'][y] <= dd['tax5'][y])
        scip.addCons(vars['tax6'][y] <= dd['tax6'][y])

        scip.addCons(vars['taxable_income'][y] == vars['tax0'][y] \
                                                + vars['tax1'][y] \
                                                + vars['tax2'][y] \
                                                + vars['tax3'][y] \
                                                + vars['tax4'][y] \
                                                + vars['tax5'][y] \
                                                + vars['tax6'][y] \
                                                + vars['tax7'][y])

        scip.addCons(vars['taxable_income'][y] == vars['ncgt0'][y] + vars['ncgt15'][y] + vars['ncgt20'][y])
        scip.addCons(vars['capgains'][y] + vars['taxable_income'][y] == vars['cgt0'][y] + vars['cgt15'][y] + vars['cgt20'][y])
        scip.addCons(vars['cgt0'][y] <= dd['cgt0'][y])
        scip.addCons(vars['cgt15'][y] <= dd['cgt15'][y])
        scip.addCons(vars['ncgt0'][y] <= vars['cgt0'][y])
        scip.addCons(vars['ncgt15'][y] <= vars['cgt15'][y])
        scip.addCons(vars['ncgt20'][y] <= vars['cgt20'][y])

        scip.addCons(vars['income_tax'][y] == 0 * vars['tax0'][y] \
                                            + 0.10 * vars['tax1'][y] \
                                            + 0.12 * vars['tax2'][y] \
                                            + 0.22 * vars['tax3'][y] \
                                            + 0.24 * vars['tax4'][y] \
                                            + 0.32 * vars['tax5'][y] \
                                            + 0.35 * vars['tax6'][y] \
                                            + 0.37 * vars['tax7'][y] \
                                            + 0.15 * vars['cgt15'][y] \
                                            + 0.22 * vars['cgt20'][y])

        scip.addCons(vars['afterTax'][y] == rors * vars['afterTax'][y-1] - vars['from_aTax'][y] + vars['to_aTax'][y])
        scip.addCons(vars['e_Roth'][y] == rori * vars['e_Roth'][y-1] - vars['from_eRoth'][y] + vars['e_RothConv'][y])
        scip.addCons(vars['e_Taxd'][y] == rori * vars['e_Taxd'][y-1] - vars['e_RMD'][y] - vars['from_eTaxd'][y] - vars['e_RothConv'][y])
        scip.addCons(vars['j_Roth'][y] == rori * vars['j_Roth'][y-1] - vars['from_jRoth'][y] + vars['j_RothConv'][y])
        scip.addCons(vars['j_Taxd'][y] == rori * vars['j_Taxd'][y-1] - vars['j_RMD'][y] - vars['from_jTaxd'][y] - vars['j_RothConv'][y])

        # For "reasons," the optimizer moves all money into afterTax in the last year of the plan.
        # Until I figure out why, or discover a constraint to prevent that, the 0.999... hack...
        scip.addCons(vars['net_pretax'][y] \
                        == (0.999999 * vars['afterTax'][y] + vars['e_Taxd'][y] + vars['j_Taxd'][y] \
                            + vars['e_Roth'][y] + vars['j_Roth'][y]))

        # For the postax value, assume the IRA beneficiaries will be in the 22% marginal tax bracket -- TODO: if it's larger than model tax rate watch out!
        #
        scip.addCons(vars['net_postax'][y] \
                        == 0.99 * vars['afterTax'][y] + vars['e_Roth'][y] + vars['j_Roth'][y] \
                            + 0.99 * (vars['e_Taxd'][y] + vars['j_Taxd'][y])) # 0.78
    #
    scip.optimize()

    OUTS = [
        'afterTax',
        'aTax_basis',
        'e_Roth',
        'e_Taxd',
        'j_Roth',
        'j_Taxd',
        'e_RMD',
        'j_RMD',
        'from_eRoth',
        'from_jRoth',
        'from_eTaxd',
        'from_jTaxd',
        'from_aTax',
        'e_RothConv',
        'j_RothConv',
        'auto_income',
        'taxable_income',
        'dividends',
        'capgains',
        'income_reqd',
        'income_tax',
        'net_pretax',
        'net_postax']

    for n in OUTS:
        for y in range(1,YRS+1):
            v = scip.getVal(vars[n][y])
            dd[n][y] = lop_to_cents(v)

    dd['income_reqd'][0] = lop_to_cents(scip.getVal(vars['income_reqd'][0])) # set if maximzing spend
    
    for y in range(1,YRS+1):
        dd['tax_bracket'][y] = \
            0.32 if lop_to_cents(scip.getVal(vars['tax5'][y])) != 0 else \
            0.24 if lop_to_cents(scip.getVal(vars['tax4'][y])) != 0 else \
            0.22 if lop_to_cents(scip.getVal(vars['tax3'][y])) != 0 else \
            0.12 if lop_to_cents(scip.getVal(vars['tax2'][y])) != 0 else \
            0.10 if lop_to_cents(scip.getVal(vars['tax1'][y])) != 0 else \
            0.00
        dd['cgains_rate'][y] = \
            0.20 if lop_to_cents(scip.getVal(vars['cgt20'][y])) != 0 else \
            0.15 if lop_to_cents(scip.getVal(vars['cgt15'][y])) != 0 else \
            0.00
        dd['from_aTax'][y] = round(dd['from_aTax'][y] - scip.getVal(vars['to_aTax'][y]), 5)

    with err_out:
        # print(f'npo {scip.getVal(vars["net_postax"][YRS])}')
        print(f'obj {scip.getObjVal()}')
        print(f'sec {scip.getSolvingTime()}')
    
    return (dd['net_pretax'][YRS], dd['net_postax'][YRS])
    

############################

# The big general purpose output for tables, graphs, etc.
out_box = widgets.Output(layout={'border': '1px solid black'})

# intput widget for file name for saving Exploration dataframe
efname = widgets.Text(
    value='data/explore.csv',
    placeholder='data/explore.csv',
    description='Output to:',
    disabled=False
)

def oorp(ssab, roth, objt, ftab):
    """Run the projection based on widget inputs and display results"""
    err_out.clear_output() # at start of each run
    out_box.clear_output()
    do_roth_convs = 1.0 if roth else 0.0
    dd = make_planning_datadict(ssab) # reduce SSA benefits after 2035
    # the projection...
    (net_pretax, net_postax) = oorplp(dd, do_roth_convs, objt, ftab)
    #
    with out_box:
        print(f'Net portfolio pre-tax: {net_pretax: 7.3f} estimated post-tax: {net_postax: 7.3f}')
        df = pd.DataFrame(dd, index = dd["year"])
        df.to_csv(efname.value)
        # barmode='relative' is same as barmode='stack' except that negative values plot below the x-axis
        display(px.bar(df, barmode='relative', x='year',
                    y=['afterTax', 'e_Taxd', 'j_Taxd', 'e_Roth', 'j_Roth'],
                    color_discrete_map={'afterTax':'royalblue', 'e_Taxd':'mediumorchid', 'j_Taxd':'mediumpurple',
                                        'e_Roth':'forestgreen', 'j_Roth':'lawngreen'},
                    title='Nominal Balances'))
        display(px.bar(df, barmode='relative', x='year', # indianred or firebrick for brick?
                    y=['SSA_income','e_RMD','j_RMD','from_eTaxd','from_jTaxd','from_aTax','from_eRoth','from_jRoth'],
                    color_discrete_map={'SSA_income':'goldenrod','e_RMD':'firebrick','j_RMD':'chocolate',
                                        'from_eTaxd':'mediumorchid','from_jTaxd':'mediumpurple',
                                        'from_aTax':'royalblue','from_eRoth':'forestgreen','from_jRoth':'lawngreen'},
                    title='Nominal Withdrawals'))
        # combine (from_eRoth + from_jRoth), (e_RMD + j_RMD + from_eTaxd + from_jTaxd)
        df['from_Roth'] = df[['from_eRoth', 'from_jRoth']].sum(axis=1)
        df['from_Taxd'] = df[['e_RMD', 'j_RMD', 'from_eTaxd', 'from_jTaxd']].sum(axis=1)
        display(px.bar(df, barmode='relative', x='year',
                    y=['SSA_income', 'from_aTax', 'from_Roth', 'from_Taxd'],
                    color_discrete_map={'SSA_income':'goldenrod','from_aTax':'royalblue',
                                        'from_Roth':'forestgreen','from_Taxd':'mediumorchid'},
                    title='Nominal Withdrawals'))
        # Tax Data
        df['income_tax'] = -df['income_tax']
        df['Roth_conv'] = df[['e_RothConv', 'j_RothConv']].sum(axis=1)
        df['wthd_Taxd'] = df[['from_eTaxd', 'from_jTaxd']].sum(axis=1)
        df['total_RMD'] = df[['e_RMD', 'j_RMD']].sum(axis=1)
        display(px.bar(df, barmode='relative', x='year',
                       y=['income_tax', 'Roth_conv', 'wthd_Taxd', 'total_RMD', 'SSA_income', 'dividends'],
                       color_discrete_map={'income_tax':'firebrick', 'Roth_conv':'forestgreen',
                                           'wthd_Taxd':'mediumorchid', 'total_RMD':'chocolate',
                                           'SSA_income':'goldenrod', 'dividends':'royalblue'},
                       title='Tax Data'))
        # Tables
        # Nominal Balances
        display(Markdown('\n### Nominal Balances'))
        display(pd.DataFrame(df, columns=['e', 'j', 'afterTax', 'aTax_basis', 'e_Roth', 'j_Roth',
                                           'e_Taxd', 'j_Taxd', 'net_pretax', 'net_postax']))
        # Nominal Withdrawals
        display(Markdown('\n### Nominal Withdrawals'))
        display(pd.DataFrame(df, columns=['e', 'j', 'e_RMD', 'j_RMD', 'from_eTaxd', 'from_jTaxd',
                                          'from_aTax', 'from_eRoth', 'from_jRoth']))
        # Tax info
        display(Markdown('\n### Tax Info'))
        display(pd.DataFrame(df, columns=['Roth_conv', 'wthd_Taxd', 'total_RMD', 'SSA_income',
                                          'dividends', 'capgains', 'misc_income',
                                          'taxable_income', 'income_tax', 'tax_bracket', 'cgains_rate', 'income_reqd']))

winter = widgets.interactive(oorp,
        ssab = widgets.Checkbox(value=True,
                description='SSA benefits are reduced 23% in 2035',
                disabled=False,
                indent=False),
        roth = widgets.Checkbox(value=True,
                description='Roth conversions',
                disabled=False,
                indent=False),
        objt = widgets.Dropdown(
                options=[('Max Residual (FTAB)', 'net_pretax'), ('Max spend', 0)],
                value='net_pretax',
                disabled=False,
                description='Objective:'),
        ftab = widgets.BoundedFloatText(
                value=0.0,
                min=0,
                max=9999.9,
                step=1.0,
                description='Final Total Account Balance',
                disabled=False)
    )

run_button = widgets.Button(description='Run Projection', disabled=False,)
run_button.on_click(winter.update)

winter_lbl = widgets.Label('e-ORP Explorer',style={ 'font_weight':'bold', 'font_size':'large',})

display(widgets.VBox([winputs,
                      err_out,
                      widgets.VBox([winter_lbl, winter, run_button, efname],
                                    layout=widgets.Layout(border='1px solid black')), # width='100%', 

                     out_box]))
